# Bounding Box Priors Estimation using K-Means

SSD uses predefined bounding boxes ratios called priors to detect objects in the grids. To improve the performance of the model, a good practice is to estimate the aspect ratios of the objects included in your custom dataset. In this ntoebook, we will analyze them using K-Means over the width and height of the boxes.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns
import os
import cv2
import xml.etree.ElementTree as ET

from sklearn.cluster import KMeans, DBSCAN
from bounding_box_utils.bboxes_parser import parse_bboxes
from jupyterthemes import jtplot

jtplot.style(theme='chesterish', grid=False)

%matplotlib inline

In [ ]:
annotations_dir = "../dataset_new/annotations/PASCAL_VOC/"
bboxes_w_h = []
classes = ['screw_0', 'screw_1', 'screw_2', 'screw_3',
           'screw_4', 'screw_5', 'screw_6', 'screw_7',
           'screw_8', 'screw_9']

for f in os.listdir(annotations_dir):
    ann = ET.parse(annotations_dir + f)
    bboxes = parse_bboxes(ann, classes)
    for bb in bboxes:
        bboxes_w_h.append(np.array([bb[2]-bb[0], bb[3] - bb[1], bb[4]]))

bboxes_w_h = np.array(bboxes_w_h)
    

## Analysis of Bounding Boxes

In [ ]:
sns.jointplot(x=bboxes_w_h[:,0], y=bboxes_w_h[:,1])
sns.jointplot(x=bboxes_w_h[:,0], y=bboxes_w_h[:,1], kind='kde')

## Plot the boxes by size

In the following histogram we can see the number of boxes per interval of sizes of the maximum between the 2 dimensions

In [ ]:
def count_base_size(bboxes, input_array):
    
    result = {}
    for ele in input_array:
        result[str(ele)] = 0
    result['rest'] = 0

    for bb in bboxes:
        max_d = max(bb[0], bb[1])        
        if max_d > input_array[len(input_array)-1]:
            result['rest'] += 1
        else:
            idx = np.argmin(np.abs(input_array-max_d))
            bound = input_array[idx]
            if max_d > bound:
                result[str(input_array[idx + 1])] += 1
            else:
                result[str(bound)] += 1
            
    return result
    
D = count_base_size(bboxes_w_h, np.array([64,96,128,196,212,256,364,448,512]))
plt.bar(range(len(D)), D.values(), align='center') 
plt.xticks(range(len(D)), D.keys()) 

plt.show()

## Compute the centroids of the k desired clusters


### K-Means on the aspect ratio

In [ ]:
# Plotting 5 clusters based on the aspect ratios

ratios = bboxes_w_h[:,0] / bboxes_w_h[:,1]
K = KMeans(5, random_state=1) 
labels = K.fit(ratios.reshape(-1,1))

print("Aspect Ratios:\n")
print(labels.cluster_centers_)

plt.plot(ratios, 'o')


### K-Means on width and height

In [ ]:
# Plotting clusters based on both width and height

K = KMeans(5, random_state=0)
labels = K.fit(bboxes_w_h[:,:2])
plt.scatter(bboxes_w_h[:,0], bboxes_w_h[:,1], c=labels.labels_,s=50, cmap='viridis');

out = labels.cluster_centers_
ar = out[:,0] / out[:,1]
scale = out[:,1]*np.sqrt(ar)/256

print ("Aspect Ratios:")
print (ar)

print ("Scales: ")
print (scale)